In [43]:
import requests
import pandas as pd
from time import sleep

base_url = "https://www.datos.gov.co/resource/jdjx-qer4.json"
limit = 50000
offset = 0
total_rows = 200000
data = []
max_retries = 3

while offset < total_rows:
    params = {
       'departamen': 'Cesar',  # Clave corregida
        '$limit': limit,
        '$offset': offset
    }

    print(f"Descargando registros desde {offset} hasta {offset + limit}...")

    for attempt in range(max_retries):
        try:
            response = requests.get(base_url, params=params, timeout=30)  # Timeout extendido a 30 segundos
            if response.status_code == 200:
                batch = response.json()
                break
            else:
                print(f"Error {response.status_code} en intento {attempt + 1}, reintentando en 5 segundos...")
                sleep(5)
        except requests.exceptions.Timeout:
            print(f"Timeout en intento {attempt + 1}, reintentando en 5 segundos...")
            sleep(5)
        except requests.exceptions.RequestException as e:
            print(f"Error en intento {attempt + 1}: {e}")
            sleep(5)
    else:
        print("Error persistente, abortando descarga.")
        break

    if not batch:
        print("No hay más datos, finalizando descarga.")
        break

    data.extend(batch)

    if len(batch) < limit:
        print("Último lote recibido, terminando.")
        break

    offset += limit

    sleep(2)  # Espera entre lotes para no saturar el servidor

df = pd.DataFrame(data)
print(f"\nDataFrame creado exitosamente con {len(df)} filas.")
print(df.head())

df.to_csv("datos_completos.csv", index=False)
print("\nDatos guardados en 'datos_completos.csv'.")

Descargando registros desde 0 hasta 50000...
Último lote recibido, terminando.

DataFrame creado exitosamente con 4665 filas.
                                            the_geom        municipio  \
0  {'type': 'MultiPolygon', 'coordinates': [[[[-7...  Agustín Codazzi   
1  {'type': 'MultiPolygon', 'coordinates': [[[[-7...  Agustín Codazzi   
2  {'type': 'MultiPolygon', 'coordinates': [[[[-7...  Agustín Codazzi   
3  {'type': 'MultiPolygon', 'coordinates': [[[[-7...        Becerrill   
4  {'type': 'MultiPolygon', 'coordinates': [[[[-7...        Becerrill   

  departamen cod_depart cod_dane_m gridcode       aptitud            area_ha  \
0      Cesar         20      20013        0       No apta  0.000241981129641   
1      Cesar         20      20013        0       No apta    0.0240969705444   
2      Cesar         20      20013        0       No apta      7783.61515207   
3      Cesar         20      20045        0       No apta    0.0176904453182   
4      Cesar         20      20045 

In [44]:
import geopandas as gpd
from shapely.geometry import shape

# Convertir 'the_geom' a geometría
df['geometry'] = df['the_geom'].apply(shape)

# Crear GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry='geometry', crs="EPSG:4326")

# Filtrar solo "Aptitud alta" y "Aptitud media"
gdf = gdf[gdf['aptitud'].isin(['Aptitud alta', 'Aptitud media'])]

# Simplificar geometrías
gdf['geometry'] = gdf['geometry'].simplify(tolerance=0.0005, preserve_topology=True)

# Eliminar columnas innecesarias
gdf = gdf[['municipio', 'departamen', 'aptitud', 'area_ha', 'geometry']]

# Guardar a archivo .geojson filtrado
gdf.to_file("datos_aptitud_alta_media.geojson", driver='GeoJSON')

print("Archivo 'datos_aptitud_alta_media.geojson' exportado exitosamente.")
import json
import folium

m = folium.Map(location=[4.5709, -74.2973], zoom_start=5)

# Colores
aptitud_color = {
    'Aptitud media': '#0000FF',
    'Aptitud alta': '#008000',
}

def style_function(feature):
    aptitud = feature['properties']['aptitud']
    color = aptitud_color.get(aptitud, 'gray')
    style = {'fillOpacity': 0.6, 'weight': 1, 'color': color, 'fillColor': color}
    return style

with open('datos_aptitud_alta_media.geojson', encoding='utf-8') as f:
    geojson_data = json.load(f)

folium.GeoJson(
    geojson_data,
    name='Aptitud',
    style_function=style_function,
    tooltip=folium.GeoJsonTooltip(fields=['municipio', 'departamen', 'aptitud', 'area_ha'],
                                  aliases=['Municipio', 'Departamento', 'Aptitud', 'Área (ha)'],
                                  localize=True)
).add_to(m)

folium.LayerControl().add_to(m)

m

Archivo 'datos_aptitud_alta_media.geojson' exportado exitosamente.


In [45]:
m.save("Cesar.html")